In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Define_Basic_Parameters

In [16]:
# define the num which represent the true number of verified sample-point sets
num = 5

# define the tree_num used as key points that record the change of accuracy
tree_num = [1] + list(range(10,121,10))

# define the year range
year_start = [f'{i}-01-01' for i in range(1987,2020,3)]
year_end   = [f'{i}-12-31' for i in range(1989,2020,3)]

year_range = list(zip(year_start,year_end))[-num:][::-1]

year_name_all  = list([f'{span[0]}_{span[1]}' for span in zip(range(1987,2020,3),range(1989,2020,3)) ])
year_name = year_name_all[-num:][::-1]

In [17]:
year_range

[('2017-01-01', '2019-12-31'),
 ('2014-01-01', '2016-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2005-01-01', '2007-12-31')]

In [18]:
year_name

['2017_2019', '2014_2016', '2011_2013', '2008_2010', '2005_2007']

##### Classify Sample_with_img_value

In [19]:
path = 'users/wangjinzhulala/North_China_Plain_Python/Sample_with_Landsat_Fourier_Nomalized_img_value'

Sample_with_value = [ee.FeatureCollection(f'{path}/Sample_{year}_extract_Landsat_Fourier_Normalized_img')
                    for year in year_name]

In [20]:
# Initiate the accuracy dictionary
Combo_instance = {}

for fe,year in  zip(Sample_with_value,year_name):
    
    # _________________________________prepare the band names_________________________________
    
    # use the first element to get all band name
    bands = list(fe.first().getInfo()['properties'].keys())
    
    # get the Landsat_band
    Landsat_re   = re.compile(r'^B\d')
    Landsat_band = list(filter(Landsat_re.match,bands))

    # get the Fourier_band
    Fourier_re   = re.compile(r'^EVI|NDBI|NDVI')
    Fourier_band = list(filter(Fourier_re.match,bands))
    
    # get the Mean_Normalized band
    Mean_re         = re.compile(r'^Mean')
    Mean_Normalized = list(filter(Mean_re.match,bands))
    
    # ________________________Create sample classification instaces_____________________________
    
    # create band_combinations
    band_combination = [Landsat_band,
                        Fourier_band,
                        Landsat_band + Fourier_band,
                        Landsat_band + Mean_Normalized,
                        Fourier_band + Mean_Normalized,
                        Landsat_band + Fourier_band + Mean_Normalized]
    
    # create comno_names
    combination_name = ['Landsat',
                        'Fourier',
                        'Landsat_Fourier',
                        'Landsat_Mean',
                        'Fourier_Mean',
                        'Landsat_Fourier_Mean']
    
    # test the accuracy of each combo
    for combo,name in zip(band_combination,combination_name):
               
        for tree in tree_num:
            
            # Instatiate the class with a name
            Accuracy_assess = Classification( year_name      = year,
                                              Input_img      = 0,
                                              input_band     = combo,
                                              Verified_point = fe,
                                              Tree_num       = tree)
            
            # add accuracy to the dict
            Accuracy_assess.Stp_2_Classification_on_Samples()
            
            # get the accuracy value
            Combo_instance[(year,name,tree)] = Accuracy_assess.Test_sample_classification

In [21]:
# innitialize the accuracy dict
Combo_acc = {}

# innitilize operation flag to report the process
Opts_flat  = 0
Opts_tatal = len(year_name) * len(combination_name) * len(tree_num)

# unpack the accuracy_instances
for year in year_name:
    for name in combination_name:
        for tree in tree_num:
            
            Opts_flat+=1
            
            Combo_acc[(year,name,tree)] = Accuracy_assesment(Combo_instance[(year,name,tree)])\
                                          .Stp_1_Calculate_Accuracy()
            # print out the process
            print(f'{year}_{name}_{str(tree).zfill(3)} accuracy computation finished! ==>{Opts_flat}/{Opts_tatal}')
        print('_________________')

2017_2019_Landsat_1 accuracy computation finished! ==>1/390
2017_2019_Landsat_10 accuracy computation finished! ==>2/390
2017_2019_Landsat_20 accuracy computation finished! ==>3/390
2017_2019_Landsat_30 accuracy computation finished! ==>4/390
2017_2019_Landsat_40 accuracy computation finished! ==>5/390
2017_2019_Landsat_50 accuracy computation finished! ==>6/390
2017_2019_Landsat_60 accuracy computation finished! ==>7/390
2017_2019_Landsat_70 accuracy computation finished! ==>8/390
2017_2019_Landsat_80 accuracy computation finished! ==>9/390
2017_2019_Landsat_90 accuracy computation finished! ==>10/390
2017_2019_Landsat_100 accuracy computation finished! ==>11/390
2017_2019_Landsat_110 accuracy computation finished! ==>12/390
2017_2019_Landsat_120 accuracy computation finished! ==>13/390
_________________
2017_2019_Fourier_1 accuracy computation finished! ==>14/390
2017_2019_Fourier_10 accuracy computation finished! ==>15/390
2017_2019_Fourier_20 accuracy computation finished! ==>16/39

KeyboardInterrupt: 

In [22]:
Combo_acc_df = pd.DataFrame.from_dict(Combo_acc).T

In [23]:
Combo_acc_df

Overall_ACC  Precisioin_non_built  Precisioin_built  \
2017_2019 Landsat 1          86.92                 92.06             49.73   
                  10         90.85                 92.22             74.79   
                  20         91.39                 93.04             74.63   
                  30         91.59                 92.99             76.74   
                  40         91.52                 92.92             76.56   
                  50         91.62                 93.05             76.63   
                  60         91.72                 93.19             76.69   
                  70         91.76                 93.19             76.98   
                  80         91.82                 93.13             77.99   
                  90         91.86                 93.13             78.29   
                  100        91.76                 93.13             77.39   
                  110        91.79                 93.07             78.12   
                  120        91.79                 93.10             77.91   
          Fourier 1          80.94                 89.39             28.09   
                  10         87.68                 88.16             66.15   
                  20         87.92                 88.11             76.92   
                  30         87.98                 88.20             76.79   
                  40         87.75                 88.14             68.85   
                  50         87.95                 88.22             74.58   
                  60         87.92                 88.19             74.14   
                  70         87.92                 88.16             75.00   
                  80         87.98                 88.17             77.78   
                  90         87.98                 88.20             76.79   
                  100        87.85                 88.13             73.21   
                  110        87.95                 88.19             75.44   
                  120        87.95                 88.19             75.44   

                       Recall_non_built  Recall_built  
2017_2019 Landsat 1               92.98         46.41  
                  10              97.74         44.87  
                  20              97.39         51.28  
                  30              97.70         50.77  
                  40              97.70         50.26  
                  50              97.66         51.28  
                  60              97.62         52.31  
                  70              97.66         52.31  
                  80              97.81         51.79  
                  90              97.85         51.79  
                  100             97.74         51.79  
                  110             97.85         51.28  
                  120             97.81         51.54  
          Fourier 1               88.60         29.74  
                  10              99.16         11.03  
                  20              99.54         10.26  
                  30              99.50         11.03  
                  40              99.27         10.77  
                  50              99.42         11.28  
                  60              99.42         11.03  
                  70              99.46         10.77  
                  80              99.54         10.77  
                  90              99.50         11.03  
                  100             99.42         10.51  
                  110             99.46         11.03  
                  120             99.46         11.03

In [24]:
verified_pt_img_value = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Sample_with_Landsat_Fourier_Nomalized_img_value/Sample_2017_2019_extract_Landsat_Fourier_Normalized_img")

In [25]:
random_pt = verified_pt_img_value.randomColumn('random')

In [37]:
list(train.limit(1).getInfo()['columns'].keys())

['B1',
 'B10',
 'B11',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'Built',
 'EVI_constant',
 'EVI_cos_1',
 'EVI_cos_2',
 'EVI_cos_3',
 'EVI_sin_1',
 'EVI_sin_2',
 'EVI_sin_3',
 'EVI_t',
 'Mean_EVI',
 'Mean_NDBI',
 'Mean_NDVI',
 'NDBI_constant',
 'NDBI_cos_1',
 'NDBI_cos_2',
 'NDBI_cos_3',
 'NDBI_sin_1',
 'NDBI_sin_2',
 'NDBI_sin_3',
 'NDBI_t',
 'NDVI_constant',
 'NDVI_cos_1',
 'NDVI_cos_2',
 'NDVI_cos_3',
 'NDVI_sin_1',
 'NDVI_sin_2',
 'NDVI_sin_3',
 'NDVI_t',
 'random',
 'system:index']

In [26]:
train = random_pt.filterMetadata('random',"not_less_than",0.3)
test  = random_pt.filterMetadata('random',"less_than",0.3)

In [38]:
classifier = ee.Classifier.smileRandomForest(100).train(features = train, 
                                                        classProperty = 'Built',
                                                       inputProperties = ['B1', 'B10', 'B11', 'B2', 'B3', 
                                                                          'B4', 'B5', 'B6', 'B7', 'B8', 'B9'])

In [52]:
t = [(i['properties']['Built'],i['properties']['classification']) for i in train.limit(5000).classify(classifier).getInfo()['features'] if i['properties']['Built'] == 1]

In [53]:
sum([j[1] for j in t])/len(t)

0.10869565217391304